## Parsing and Structuring Time-Series from NASDAQ

Here the goal is to combine and tidy the individual time-series obtained from NASDAQ along with relevant metadata for further analysis.

In [2]:
import pandas as pd
import numpy as np
import json
import os 
from tqdm import tqdm

with open("config.json","r") as f:
    config = json.load(f)
    
# Read list of files    
list_of_data = os.listdir(config["data_dir"])
list_of_df = [pd.read_csv(config["data_dir"] + "/" + x)\
                                        for x in list_of_data]

for i in tqdm(range(len(list_of_data))): 
    list_of_df[i]["Symbol"] = list_of_data[i][:-4] # Add ticker symbols
    list_of_df[i]['Date'] = pd.to_datetime(list_of_df[i]['Date'], # Format date column
                                          infer_datetime_format=True)
    try:
        list_of_df[i]['Close/Last'] = pd.to_numeric(list_of_df[i]['Close/Last'].str.replace("\\$",""))
        list_of_df[i]['Open'] = pd.to_numeric(list_of_df[i]['Open'].str.replace("\\$",""))
        list_of_df[i]['High'] = pd.to_numeric(list_of_df[i]['High'].str.replace("\\$",""))
        list_of_df[i]['Low'] = pd.to_numeric(list_of_df[i]['Low'].str.replace("\\$",""))
    except:
        pass
    
ts_data = pd.concat(list_of_df)# Rbind
# Add meta data
ts_data = pd.merge(ts_data,
                   pd.read_csv(config['metadata'])[['Symbol',
                                                    'Name',
                                                    'Market Cap',
                                                    'Country',
                                                    'IPO Year',
                                                    'Sector',
                                                    'Industry'
                                                   ]]
                   ,on = "Symbol")

100%|█████████████████████████████████████████████████████████████████████████████| 3833/3833 [00:25<00:00, 144.16it/s]


In [3]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2870346 entries, 0 to 2870345
Data columns (total 13 columns):
Date          datetime64[ns]
Close/Last    float64
Volume        float64
Open          float64
High          float64
Low           float64
Symbol        object
Name          object
Market Cap    float64
Country       object
IPO Year      float64
Sector        object
Industry      object
dtypes: datetime64[ns](1), float64(7), object(5)
memory usage: 306.6+ MB


In [4]:
# Unique sectors in the data
print (ts_data['Sector'].unique())
# Unique industries
print(ts_data['Industry'].unique())

[nan 'Health Care' 'Transportation' 'Finance' 'Technology' 'Capital Goods'
 'Consumer Durables' 'Miscellaneous' 'Basic Industries'
 'Consumer Services' 'Public Utilities' 'Energy' 'Consumer Non-Durables']
[nan 'Biotechnology: Pharmaceutical Preparations'
 'Air Freight/Delivery Services' 'Life Insurance' 'Semiconductors'
 'Industrial Machinery/Components' 'Computer Manufacturing'
 'Transportation Services' 'Major Banks' 'Business Services'
 'Biotechnology: In Vitro & In Vivo Diagnostic Substances'
 'Medical/Dental Instruments'
 'Biotechnology: Biological Products (No Diagnostic Substances)'
 'Hospital/Nursing Management' 'EDP Services' 'Specialty Chemicals'
 'Specialty Insurers' 'Multi-Sector Companies'
 'Computer Software: Prepackaged Software' 'Industrial Specialties'
 'Telecommunications Equipment' 'Medical/Nursing Services' 'Advertising'
 'Electrical Products' 'Building operators'
 'Diversified Commercial Services' 'Electric Utilities: Central'
 'Environmental Services' 'Television 

In [20]:
# Date range
print(ts_data['Date'].max(),"-",ts_data['Date'].min())

2021-12-10 00:00:00 - 2016-12-05 00:00:00


In [ ]:
from tqdm import tqdm
# add % valuation change for a given ticker
tickers = ts_data['Symbol'].unique()
def calculate_perc_change(ticker):
    x = ts_data[ts_data['Symbol'] == ticker]
    price_diff = -1 * np.diff(x['Close/Last'])
    price_diff_perc = 100 * (price_diff / x['Close/Last'][1:])
    price_diff_perc = np.append(price_diff_perc,0)
    x['price_diff_perc'] = price_diff_perc
    return x
ts_data_expand = [calculate_perc_change(ticker) for ticker in tqdm(tickers)]
ts_data_expand = pd.concat(ts_data_expand)    

  0%|                                                                                         | 0/3822 [00:00<?, ?it/s]C:\Users\OZAN\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
 40%|██████████████████████████████▉                                               | 1513/3822 [06:07<08:58,  4.29it/s]

ModuleNotFoundError: No module named 'tqdm'

In [50]:
tickers[1234]

'EYPT'

In [1]:
%qtconsole